## RetinaNet 
- Paper : (2017) Focal Loss for Dense Object Detection
- **기존의 One-stage Dense Detection 알고리즘 분류기에서의 Loss Function으로 사용되던 CE(cross entropy) ->  간단한 변형이 된 Focal Loss를 제안**
- RetinaNet은 one-stage detector로 판단속도가 빠르고, two-stage detector보다 정확도가 높음

![retinanet_graph](img/retinanet_graph.png)

## RetinaNet 배경
- Two-stage 기반 : 높은 정확도를 지님 / Classifier가 Sparse한 후보영역을 생성
- One-stage 기반 : 처리속도 빠름 / 정확도가 낮은편 / dense detector / **전경과 배경이 극단적으로 불균형(imbalance)가 낮은 정확도의 원인**
- **전경과 배경과의 class imbalance 문제를 해결하기 위해, 잘 분류된 샘플에 대해 가중치를 낮추는 방식으로 변형** 방법 제안
- 즉 Focal Loss는 적은 수의 hard examples이 주어진 상태에서 많은 easy negatives sample을 지배적으로 학습해버리는 상황을 예방하는데 중점 **'Easy Negative Sample이 모델을 압도해버리는 문제'**

<hr>



## One stage vs Two stage
- One-stage detector
    - **Region proposal 없이, CNN Feature map에서 localization을 진행**
    - 이를 위해 Anchor Box(미리 정의해서 계산 효율성 증가), Scale, aspect ratio(anchor box 가로세로비율) 등을 적용
    - **Feature Map의 grid마다 anchor box를 적용하기에, one-stage보다 candidate object locations의 갯수가 훨씬 많음 (Object 갯수 < Foreground 갯수)**
   
- Two stage detector
    - First stage에서 object가 존재할 확률이 높은 region proposal을 생성하기에 class 불균형 문제가 덜 민감
    
<hr>

## Focal Loss

### Balanced Cross Entropy
- class의 불균형을 해결하기 위해 Cross Entropy에 weighting factor α를 적용
- Positive/Negative example에 대한 차별성만 표현 가능

![retinanet_balanced_crossentropy](img/retinanet_balanced_crossentropy.png)


### Focal Loss Definition
- 기존의 α-balanced cross entropy loss에 + $$ (1-p_t)^γ$$ 곱을 추가한 것
- easy example에 대한 가중치를 줄인다
    - easy example에 대한 loss를 줄임
- hard example의 학습에 초점을 맞춘다.
    - hard example에 대한 loss를 키움

![retinanet_focalloss2](img/retinanet_focalloss2.png)

- (1) 어떤 샘플이 분류가 잘못되고,$$ p_t $$ 가 작을 때, 조정 factor는 1에 가까운 값을 가지며, losssms 거의 영향이 없다. 
- (2) $$p_t$$ -> 1일때, 조정 factor가 0에 가까워 지고 잘 분류된 샘플에 대한 loss는 가중치가 작아짐
- (3) focusing 파라미터인 γ는 easy 샘플들에 대한 가중치를 낮추는 정도를 somooth하게 조정
- (4) γ=0일 때에는 FL은 CE와 같으며, γ가 증가하면 modulating factor의 효과가 증가하는 것과 같음
- **즉 조정 Factor는 easy 샘플들의 loss에 대한 영향을 줄이고, 샘플이 낮은 loss를 받는 구간을 확장하여, Object 검출에 집중할 수 있게 도움**

![retinanet_focalloss](img/retinanet_focalloss.png)

<hr>

## RetinaNet Structure
- RetinaNet은 **backbone 네트워크와 두개의 task-specific subnetworks로 구성**
    - Backbone : Feature Extracting 수행
        - ResNet
        - FPN : Multi-scale conv feature pyramid -> Anchor box 생성
    - subnetworks 1) Class subnet
        - 각 anchor box의 **object가 존재할 확률 예측**
        - 3*3 conv layers, ReLU activations
    - subnetworks 2) Box subnet
        - anchor box와 GT box를 비교 및 **bounding box를 추정** 
        - 앵커 박스의 offset 4개(box_x_center, box_y_center, box_width, box_height)를 GT박스와 유사하게 regression 함
        - class-agnostic bounding box regressor를 사용 
        - class subnet과 box subnet은 구조는 같지만 개별적인 파라미터를 사용 (파라미터 공유 x)
![retinanet_structure](img/retinanet_structure.png)

## RetinaNet result - 성능비교

![retinanet_result](img/retinanet_result.png)

## Focal loss 요약
- 기존 One-stage의 정확도가 낮은 문제를 각 grid마다 Anchor Box들을 만드는데, Object보다 배경이 많이 항상 포함되는 문제 때문이라고 봄. (Two stage는 Region Proposal을 통해 Object가 있을만한 영역을 선 제안하기 떄문에 One-stage보다 덜함)
- 이를 개선하기 위해 CE에서 Focal Loss라는 학습되기 쉬운 easy샘플들(배경)들이 주는 loss영향을 줄이고, 어려운 Hard샘플들, 보다 정밀한 Object 검출에 집중할 수 있게 도움.
- RetinaNet : ResNet + FPN + Class Subnet(object 존재여부) + Box Subnet(box 좌표들 위치 조정)

<hr>

## 참고문헌
- https://uk-kim.github.io/2018/12/07/Focal-loss-for-dense-object-detection.html
- https://ropiens.tistory.com/83